## Deploy de modelo com o Triton Inference Server

## Instalando bibliotecas

In [18]:
## Libs:
!pip3 install tritonclient\[all\]

In [42]:
#!tree modelo_rede_neural
!find modelo_rede_neural -print | sed -e 's;[^/]*/;|____;g;s;____|; |;g'

modelo_rede_neural
|____.DS_Store
|____1
| |______pycache__
| | |____model.cpython-310.pyc
| |____model.py
| |____model_vec.pkl
| |____model_net
| | |____fingerprint.pb
| | |____variables
| | | |____variables.data-00000-of-00001
| | | |____variables.index
| | |____saved_model.pb
| | |____assets
|____config.pbtxt


Como o modelo utilizado foi desenvolvido com Rede Neural, a estrutura foi salva utilizando o formado SavedModel que acaba por criar uma substrutura de pastas.
Outro detalhe é que foram salvos dois modelos um para Rede Neural (SavedModel) e outro para o TFIDF (.plk)

## Config.pbxt - Protobuffer
O arquivo de de configuração do config.pbtxt especifica as entradas e saídas dos modelos.
No caso, a entrada é uma string é uma ou mais strings e a saída é uma probabilidade da forma de sring.

In [51]:
!cat modelo_rede_neural/config.pbtxt

backend: "python"

input {
    name: "ENTRADA"
    data_type: TYPE_STRING
    dims: [ 1 ]
}

output {
    name: "PREDICAO"
    data_type: TYPE_STRING
    dims: [ 1 ]
}

instance_group [{ kind: KIND_CPU }]


## Model.py 
- O arquivo model.py contém o código para carregar o modelo com base nas configurações fornecidas pelo protobuffer.

In [49]:
!cat modelo_rede_neural/1/model.py

import json
import numpy as np
import triton_python_backend_utils as pb_utils
from joblib import load
import tensorflow as tf


class TritonPythonModel:
    def initialize(self, args):
        self.model_config = model_config = json.loads(args['model_config'])

        predicao_config = pb_utils.get_output_config_by_name(
            model_config, "PREDICAO")
        
        self.predicao_dtype = pb_utils.triton_string_to_numpy(
            predicao_config['data_type'])

        version_path =  args['model_repository'] + '/' + args['model_version']

        self.vectorize = load(version_path + '/model_vec.pkl')
        self.model = tf.saved_model.load(version_path + '/model_net')
        self.model.predict = self.model.signatures['serving_default']

    def execute(self, requests):
        responses = []

        for request in requests:
            in_x = pb_utils.get_input_tensor_by_name(request, "ENTRADA")

            input_x = in_x.as_numpy()
            input_x = self.vectorize.

## Deploy do modelo na triton usando o podman

Imagem do triton mais recente: 23.09-py3
# Imagem do triton mais recente: 24.03-py3
`podman pull nvcr.io/nvidia/tritonserver:23.09-py3`

No terminal, executar o comando:

`podman run --rm -p 8000:8000 -v $HOME/Documents/repositorio_git_estudos/triton:/models nvcr.io/nvidia/tritonserver:24.03-py3 /bin/bash -c "pip install -r /models/requirements.txt && tritonserver --model-repository=/models"`


O que faz esse comado:

- Executa um contêiner usando o podman, mapeando a porta 8000 do host para a porta 8000 do contêiner e montando o diretório $HOME/Documents/repositorio_git_estudos/triton do host para /models dentro do contêiner. 
- Instala o requirements (bibliotecas necessárias) para o modelo, neste caso foram utilizadas a biblioteca tensorflow e scikit-learn.

## Após o deploy, vamos realizar inferências!!

In [1]:
import requests
import json
import numpy as np

url = "http://localhost:8000/v2/models/modelo_rede_neural/versions/1/infer"


# input data
input_data = np.array(['it is a fake news']).reshape(1)

payload = json.dumps({
  "inputs": [
    {
      "name": "ENTRADA",
      "shape": input_data.shape,
      "datatype": "BYTES",
      "data": input_data.tolist()
    }
  ]
})

headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.text)
# Map the request in Fake or Real news
response = float(json.loads(response.text)["outputs"][0]["data"][0])
response = 'Fake news' if response > .5  else  'Real news'
print(response)

/Users/williansmagalhaesprimo/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{"model_name":"modelo_rede_neural","model_version":"1","outputs":[{"name":"PREDICAO","datatype":"BYTES","shape":[1,1],"data":["0.999789834022522"]}]}
Fake news


Para verificar se o modelo estava respondendo adequadamente, foram realizados alguns acionamentos por meio da URL local enviando um string.
Após o retorno (response), é feito um mapeamento de probabilidade: Acima de 50% é considerado Fake news e abaixo é considerado uma Notícia real.

## Comandinhos de verificação
Os comandos a seguir são apenas para validar se o servidor está ativo e os modelos estão disponíveis para inferencia.

In [44]:
# Criando um cliente para se comunicar com o Triton
import tritonclient.http as httpclient

triton_client = httpclient.InferenceServerClient(url="localhost:8000", verbose=True)

In [45]:
# Verificar se o servidor está ativo para receber solicitações
triton_client.is_server_live()

GET /v2/health/live, headers {}
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>


True

In [46]:
# Verificar se o Triton está pronto para receber inferências
triton_client.is_server_ready()

GET /v2/health/ready, headers {}
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>


True

In [47]:
# Metadados do modelo 
triton_client.get_model_metadata("modelo_rede_neural")

GET /v2/models/modelo_rede_neural, headers {}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '190'}>
bytearray(b'{"name":"modelo_rede_neural","versions":["1"],"platform":"python","inputs":[{"name":"ENTRADA","datatype":"BYTES","shape":[1]}],"outputs":[{"name":"PREDICAO","datatype":"BYTES","shape":[1]}]}')


{'name': 'modelo_rede_neural',
 'versions': ['1'],
 'platform': 'python',
 'inputs': [{'name': 'ENTRADA', 'datatype': 'BYTES', 'shape': [1]}],
 'outputs': [{'name': 'PREDICAO', 'datatype': 'BYTES', 'shape': [1]}]}